# Nodes, Terminals, and Measurements

This tutorial introduces how nodes and terminals are used in the Common Information Model to describe connectivity of power system equipment to the electrical network.

__Learning Objectives:__

At the end of the tutorial, the user should be able to 

* Explain what is a `ConnectivityNode`
* Explain what is a `Terminal`
* Describe how various types of power system equipment are connected through `Terminal` and `ConnectivityNode` objects
* List common one-terminal and two-terminal devices
* Query for Connectivity Nodes
* Query for Terminals


## Getting Started

Before running any of the sample routines in this tutorial, it is first necessary to start the GridAPPS-D Platform and establish a connection to this notebook so that we can start passing calls to the API.

_Open the Ubuntu terminal and start the GridAPPS-D Platform if it is not running already:_

`cd gridappsd-docker`

~/gridappsd-docker$ `./run.sh`

_Once containers are running,_

gridappsd@[container]:/gridappsd$ `./run-gridappsd.sh`

In [ ]:
# Establish connection to GridAPPS-D Platform:
from gridappsd import GridAPPSD
import os # Set username and password
os.environ['GRIDAPPSD_USER'] = 'tutorial_user'
os.environ['GRIDAPPSD_PASSWORD'] = '12345!'
gapps = GridAPPSD()
assert gapps.connected

---

## Table of Contents

* [1. Introduction to the PowerGrid Model API](#1.-Introduction-to-the-PowerGrid-Model-API)


* [2. API Syntax Overview](#2.-API-Syntax-Overview)
    * [2.1. API Communication Channel](#2.1.-API-Communication-Channel)
    * [2.2. Structure of a Query Message](#2.2.-Structure-of-a-Query-Message)
    * [2.3. Specifying the requestType](#2.3.-Specifying-the-requestType)
    

* [3. Querying for Feeder Model Info](#3.-Querying-for-Feeder-Model-Info)
    * [3.1. Query for mRIDs of all Models](#3.1.-Query-for-mRIDs-of-all-Models)
    * [3.2.-Query for Details Dictionary of all Models](#3.2.-Query-for-Details-Dictionary-of-all-Models)
    
    
* [4. Querying for Object Info](#4.-Querying-for-Object-Info)
    * [4.1. Query for CIM Classes of Objects in Model](#4.1.-Query-for-CIM-Classes-of-Objects-in-Model)
    * [4.2. Query for mRIDs of Objects in a Feeder](#4.2.-Query-for-mRIDs-of-Objects-in-a-Feeder)
    * [4.3. Query for CIM Attributes of an Object](#4.3.-Query-for-CIM-Attributes-of-an-Object)
    * [4.4. Query for Object Dictionary](#4.4.-Query-for-Object-Dictionary)
    
    
* [5. Querying for Object Measurements](#5.-Querying-for-Object-Measurements)
    * [5.1. Object mRIDs vs Measurement mRIDs](#5.1.-Object-mRIDs-vs-Measurement-mRIDs)
    * [5.2. Querying for Measurements](#5.2.-Querying-for-Measurements)
    * [5.3. Filtering Returned Data](#5.3.-Filtering-Returned-Data)
    
    
* [6. GridAPPSD-Python Shortcut Methods](#6.-GridAPPSD-Python-Shortcut-Methods)
    * [6.1. Querying for mRIDs of all Models](#6.1.-Querying-for-mRIDs-of-all-Models)
    * [6.2. Query for CIM Classes of Objects in Model](#6.2.-Query-for-CIM-Classes-of-Objects-in-Model)
    * [6.3. Query for CIM Attributes of an Object](#6.3.-Query-for-CIM-Attributes-of-an-Object)

---

# 1. Introduction to Nodes and Terminals

## Introduction to Nodes and Terminals

All current carrying electrical equipment inheriting from the `ConductingEquipment` class are defined as being connected to a `Terminal` object associated with a `ConnectivityNode`. 

Generators, shunt capacitors, shunt reactors, and loads are connected to one `Terminal` object associated with a single `ConnectivityNode`. 

Lines, cables, series capacitors, and series reactors have two `Terminal` objects associated with either end of the branch. 



Rather than defining from/to buses, CIM identifies the end of a branch by setting the ACDCTerminal.sequenceNumber attribute of a terminal to values of 1 or 2 to specify the particular end. Transformers are defined using two or three terminals, with the ACDCTerminal.sequenceNumber attribute used to designate whether the Terminal is associated with the primary, secondary, or tertiary windings. Split-phase distribution transformers also use three terminals, but the two low-side terminals are associated with a single ConnectivityNode, 

In [ ]:
"""
# list all the connectivity nodes (aka bus)
PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX c:  <http://iec.ch/TC57/CIM100#>
SELECT ?name WHERE {
 ?s r:type c:ConnectivityNode.
 ?s c:IdentifiedObject.name ?name}
ORDER by ?name
"""

In [ ]:
"""
# list all the connectivity node base voltages by feeder, for sensor service
PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX c:  <http://iec.ch/TC57/CIM100#>
SELECT DISTINCT ?feeder ?busname ?cnid ?nomv WHERE {
# VALUES ?fdrid {"_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"}  # 13 bus
 ?fdr c:IdentifiedObject.mRID ?fdrid.
 ?bus c:ConnectivityNode.ConnectivityNodeContainer ?fdr.
 ?bus r:type c:ConnectivityNode.
 ?bus c:IdentifiedObject.name ?busname.
 ?bus c:IdentifiedObject.mRID ?cnid.
 ?fdr c:IdentifiedObject.name ?feeder.
 ?trm c:Terminal.ConnectivityNode ?bus.
 ?trm c:Terminal.ConductingEquipment ?ce.
 ?ce  c:ConductingEquipment.BaseVoltage ?bv.
 ?bv  c:BaseVoltage.nominalVoltage ?nomv.
}
ORDER by ?feeder ?busname ?nomv
"""

In [ ]:
"""
# list all the connectivity nodes by feeder for CIMWriter
PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX c:  <http://iec.ch/TC57/CIM100#>
SELECT ?name ?id WHERE {
 VALUES ?fdrid {"_E407CBB6-8C8D-9BC9-589C-AB83FBF0826D"}  # 123 bus with PV
 ?fdr c:IdentifiedObject.mRID ?fdrid.
 ?s c:ConnectivityNode.ConnectivityNodeContainer ?fdr.
 ?s r:type c:ConnectivityNode.
 ?s c:IdentifiedObject.name ?name.
 bind(strafter(str(?s),"#") as ?id).
}
ORDER by ?name
"""

In [ ]:
"""
# list all the terminals by feeder for CIMWriter
PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX c:  <http://iec.ch/TC57/CIM100#>
SELECT ?eqclass ?eqname ?seq ?eqid ?tid ?cnid WHERE {
 VALUES ?fdrid {"_E407CBB6-8C8D-9BC9-589C-AB83FBF0826D"}  # 123 bus with PV
 ?fdr c:IdentifiedObject.mRID ?fdrid.
 ?eq c:Equipment.EquipmentContainer ?fdr.
 ?t c:Terminal.ConductingEquipment ?eq.
 ?t c:Terminal.ConnectivityNode ?cn. 
 ?t c:ACDCTerminal.sequenceNumber ?seq.
 bind(strafter(str(?eq),"#") as ?eqid).
 bind(strafter(str(?t),"#") as ?tid).
 bind(strafter(str(?cn),"#") as ?cnid).
 ?eq c:IdentifiedObject.name ?eqname.
 ?eq a ?classraw.
 bind(strafter(str(?classraw),"CIM100#") as ?eqclass)
}
ORDER by ?eqclass ?eqname ?seq
"""

In [ ]:
"""
# list all the connectivity nodes by feeder, with voltage limits
PREFIX r:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX c:  <http://iec.ch/TC57/CIM100#>
SELECT ?feeder ?bus ?cnid ?val ?dur ?dir WHERE {
# VALUES ?fdrid {"_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"}  # 13 bus
 ?fdr c:IdentifiedObject.mRID ?fdrid.
 ?s c:ConnectivityNode.ConnectivityNodeContainer ?fdr.
 ?s r:type c:ConnectivityNode.
 ?s c:IdentifiedObject.name ?bus.
 ?s c:IdentifiedObject.mRID ?cnid.
 ?fdr c:IdentifiedObject.name ?feeder.
 ?s c:ConnectivityNode.OperationalLimitSet ?ols.
 ?vlim c:OperationalLimit.OperationalLimitSet ?ols.
 ?vlim r:type c:VoltageLimit.
 ?vlim c:OperationalLimit.OperationalLimitType ?olt.
 ?olt c:OperationalLimitType.acceptableDuration ?dur.
 ?olt c:OperationalLimitType.direction ?rawdir.
  bind(strafter(str(?rawdir),"OperationalLimitDirectionKind.") as ?dir)
 ?vlim c:VoltageLimit.value ?val.
}
ORDER by ?feeder ?bus ?val
"""